In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

x = np.array([[1,1], [2,2], [3,3], [4,4], [5,5]])
y = np.array([[2,2], [3,3], [4,4]])
distance, path = fastdtw(x, y, dist=euclidean)
print(distance)
n_nodes = 10
upper_tri = torch.randint(0, 2, (n_nodes, n_nodes)) # 随机生成[0, 2)之间的整数
upper_tri = torch.triu(upper_tri, diagonal=1) # 将矩阵主对角及主对角以下元素设置为0
adj_matrix = upper_tri + upper_tri.t() + torch.eye(n_nodes)# 保证邻接矩阵对称
print(adj_matrix.dtype)

n = torch.arange(0, 20).reshape(10, 2)
m = torch.arange(0, 20).reshape(20, 1)
index = [0, 1, 2, 3]
print(n[index])
print(n.reshape(-1)[-1])
a = [i for i in range(2, 10, 2)]
print(a)
print(n.reshape(m.shape))
b = torch.arange(0, 72).reshape(6, 12) # time 6, value 4, batch 3
print(b)
print(b.reshape(-1, 4))
c = torch.cat(b.split(3, dim=0), dim=1)
print(torch.cat(c.split(12, dim=1), dim=0))
d = b.reshape(-1)
print(d)
print(d.reshape(b.shape))
e = torch.arange(0, 20)
f = torch.stack(e.split(4, dim=0), dim=1)
print(f)
print(torch.cat(f.split(1, dim=1), dim=0))

2.8284271247461903
torch.float32
tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7]])
tensor(19)
[2, 4, 6, 8]
tensor([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [14],
        [15],
        [16],
        [17],
        [18],
        [19]])
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
        [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47],
        [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
        [60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]])
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23],
        [24, 25, 26, 27],
        [28, 29, 30, 31],
        [32, 33, 34, 35],
        [36, 3

In [10]:
degree = torch.sum(adj_matrix, dim=1)
degree_inv_sqrt = degree.pow(-0.5)
degree, degree_inv_sqrt

(tensor([4., 6., 5., 8., 7., 8., 6., 7., 6., 7.]),
 tensor([0.5000, 0.4082, 0.4472, 0.3536, 0.3780, 0.3536, 0.4082, 0.3780, 0.4082,
         0.3780]))

In [11]:
A_norm = adj_matrix * degree_inv_sqrt.reshape(-1, 1)
A_norm = A_norm * degree_inv_sqrt

adj_norm = adj_matrix * degree_inv_sqrt[:, None] # degree_inv_sqrt[:, None]调整了度矩阵形状
adj_norm = adj_norm * degree_inv_sqrt[None, :]
A_norm == adj_norm

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [12]:
x = torch.randn((5, 4, 10, 5), dtype=torch.float32)
w = torch.randn((5, 8), dtype=torch.float32)
y = torch.matmul(A_norm, x)
y = torch.matmul(y, w)
y = y.reshape(5, 4, -1)
y.shape
y

tensor([[[-1.3347,  0.4799, -0.8790,  ..., -0.2809,  1.5421, -0.3082],
         [-0.0151, -0.2540,  0.8244,  ..., -0.1754,  0.7925,  0.1204],
         [ 0.5819, -0.1050,  0.6613,  ...,  0.2452,  0.0655, -0.8171],
         [-0.4976,  0.7946,  0.0657,  ...,  0.3853, -0.5544, -0.1056]],

        [[ 0.0426,  0.4874,  0.8210,  ...,  0.9566, -0.6089, -0.0787],
         [-0.2884,  0.6280, -0.0028,  ...,  0.1571, -0.5574,  1.1035],
         [ 0.6859, -0.5241,  0.2817,  ...,  0.2293, -0.5864,  0.1197],
         [-0.1644,  0.2097, -0.4034,  ...,  1.1664, -0.3955,  0.4370]],

        [[-0.1782, -0.0874,  0.5106,  ...,  0.6927,  0.0215, -1.3095],
         [-1.1741,  1.1325,  0.0699,  ..., -1.6634,  0.5199,  0.0522],
         [ 0.2421, -0.1489, -0.5400,  ...,  1.9069, -0.6936, -0.1559],
         [ 0.2902, -0.7433,  0.6532,  ...,  0.6871, -0.9531, -0.1235]],

        [[ 0.3663,  0.4436, -0.0897,  ..., -1.5657,  1.8553,  0.2667],
         [-0.5762,  0.1992, -0.1589,  ...,  2.0778, -1.0215,  1.2231],


In [13]:
def Forward(x, adj_norm, w):
        # 直接使用torch.matmul进行批量矩阵乘法
        # input: (batch_size, n_nodes, in_features) 乘以 weight: (in_features, out_features)
        support = torch.matmul(x, w)  # (batch_size, n_nodes, out_features)
        # 归一化邻接矩阵为 (n_nodes, n_nodes) 会自动广播到batch维度
        output = torch.matmul(adj_norm, support)
        return output

gcn_outputs = []
for t in range(x.size(1)):
    x_t = x[:, t, :, :]  # (batch, nodes, feat)
    h = Forward(x_t, A_norm, w)
    # 展平节点特征 (batch, nodes*hidden)
    h_flat = h.view(x.size(0), -1)
    gcn_outputs.append(h_flat.unsqueeze(1))  # 每个时间步的输出

# 构建GRU输入序列 (batch, time, features)
gru_input = torch.cat(gcn_outputs, dim=1)
gru_input.shape
gru_input

tensor([[[-1.3347,  0.4799, -0.8790,  ..., -0.2809,  1.5421, -0.3082],
         [-0.0151, -0.2540,  0.8244,  ..., -0.1754,  0.7925,  0.1204],
         [ 0.5819, -0.1050,  0.6613,  ...,  0.2452,  0.0655, -0.8171],
         [-0.4976,  0.7946,  0.0657,  ...,  0.3853, -0.5544, -0.1056]],

        [[ 0.0426,  0.4874,  0.8210,  ...,  0.9566, -0.6089, -0.0787],
         [-0.2884,  0.6280, -0.0028,  ...,  0.1571, -0.5574,  1.1035],
         [ 0.6859, -0.5241,  0.2817,  ...,  0.2293, -0.5864,  0.1197],
         [-0.1644,  0.2097, -0.4034,  ...,  1.1664, -0.3955,  0.4370]],

        [[-0.1782, -0.0874,  0.5106,  ...,  0.6927,  0.0215, -1.3095],
         [-1.1741,  1.1325,  0.0699,  ..., -1.6634,  0.5199,  0.0522],
         [ 0.2421, -0.1489, -0.5400,  ...,  1.9069, -0.6936, -0.1559],
         [ 0.2902, -0.7433,  0.6532,  ...,  0.6871, -0.9531, -0.1235]],

        [[ 0.3663,  0.4436, -0.0897,  ..., -1.5657,  1.8553,  0.2667],
         [-0.5762,  0.1992, -0.1589,  ...,  2.0778, -1.0215,  1.2231],


In [14]:
gru_input.round(decimals=6) == y.round(decimals=6)

tensor([[[ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True, False,  True]],

        [[ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ..., False,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True]],

        [[ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True]],

        [[ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ...,  True,  True,  True],
         [ True,  True,  True,  ..., False,  True,  True]],

        [[ True,  True,  True,  ...,  True,  Tru

In [15]:
m = nn.Linear(20, 30)
input = torch.randn(10, 128, 20)
print(input[[0, 1]].shape)
output = m(input)
print(output.size())
x = torch.arange(0, 10)
y = x[1]
y.unsqueeze_(dim=0)
y, len(output), (x.sum() / x.shape[0]).sqrt()

torch.Size([2, 128, 20])
torch.Size([10, 128, 30])


(tensor([1]), 10, tensor(2.1213))

In [16]:
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
loss(input, target)
y = np.array([[1, 2], [3, 4], [5, 6]])
y.shape[1]

x = torch.tensor(2)
print(x)
x.item()

tensor(2)


2

In [ ]:
q = torch.arange(0, 30).reshape(3, 2, 5)
print(q)
print(q.reshape(-1, q.shape[-1]))
print(q.transpose(1, 2))
for i in q:
    print(i)

tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9]],

        [[10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]],

        [[20, 21, 22, 23, 24],
         [25, 26, 27, 28, 29]]])
tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29]])
tensor([[[ 0,  5],
         [ 1,  6],
         [ 2,  7],
         [ 3,  8],
         [ 4,  9]],

        [[10, 15],
         [11, 16],
         [12, 17],
         [13, 18],
         [14, 19]],

        [[20, 25],
         [21, 26],
         [22, 27],
         [23, 28],
         [24, 29]]])
tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])
tensor([[10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])
tensor([[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29]])


In [18]:
label = "产物浓度"
mic_path = 'data\mic_result.xlsx'
adjmatrix = pd.read_excel(mic_path, header=0, index_col=0)
adjmatrix = adjmatrix.drop(index=label, columns=label)
adjmatrix = np.array(adjmatrix)
norm = np.zeros(adjmatrix.shape)
norm[adjmatrix > 0.4] = 1
norm

array([[1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0.],
       [0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0.],
       [0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.],
       [0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1.],
       [0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1.]])

In [19]:
q = torch.arange(0, 30, dtype=torch.float32).reshape(3, 2, 5)
state = torch.zeros((4, 3, 10), dtype=torch.float32)
net = nn.GRU(5, 10, 4, batch_first=True)
h, state = net(q, state)
h.shape, state.shape

(torch.Size([3, 2, 10]), torch.Size([4, 3, 10]))

In [10]:
def graphcov(X, adjmatrix):
    """图卷积"""
    degree = adjmatrix.sum(dim=1)
    degree_inv_sqrt = degree.pow(-0.5)
    degree_inv_sqrt[torch.isinf(degree_inv_sqrt)] = 0
    adjmatrix_norm = adjmatrix * degree_inv_sqrt.reshape(-1, 1)
    adjmatrix_norm = adjmatrix_norm * degree_inv_sqrt
    return torch.matmul(adjmatrix_norm, X)

class GCNCov(nn.Module):
    """图卷积块"""
    def __init__(self, in_features, out_features, adjmatrix, dropout=0.2):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('adjmatrix', adjmatrix)

    def forward(self, X):
        Y = graphcov(X, self.adjmatrix)
        return self.dropout(F.relu(self.linear(Y)))
    
class GCNGRU(nn.Module):
    """GCN+GRU"""
    def __init__(self, in_features, varible_num, hidden_fatures, out_features, num_layers, adjmatrix, dropout=0.2):
        super().__init__()
        self.in_features = in_features
        self.hidden_features = hidden_fatures
        self.out_features = out_features
        self.num_layers = num_layers
        self.gcn1 = GCNCov(in_features, in_features, adjmatrix, dropout)
        self.gcn2 = GCNCov(in_features, in_features, adjmatrix, dropout)
        self.gru = nn.GRU(varible_num, hidden_fatures, num_layers, 
                          batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_fatures, out_features)

    def forward(self, X, states):
        Y = self.gcn2(self.gcn1(X))
        Y, _ = self.gru(Y.transpose(1, 2), states)
        Y = self.fc(Y.reshape((-1, Y.shape[-1])))
#        Y = self.fc(Y[:, -1, :])
        return Y
    
    def init_states(self, device, batch_size):
        return torch.zeros((self.num_layers, batch_size, 
                            self.hidden_features), device=device)
win_size = 8
gru_hidden = 128
gru_layers = 2
varible_num = 14
adjmatrix = torch.ones((varible_num, varible_num), dtype=torch.float32)
dropout = 0.2
device = 'cuda:0'

X = torch.randn((50, varible_num, win_size), dtype=torch.float32, device=device)
new_net = GCNGRU(win_size, varible_num, gru_hidden, 1, gru_layers, adjmatrix, dropout)
new_net = new_net.to(device=device)
state = new_net.init_states(device, 50)
y = new_net(X, state)
print(y.shape)
for name, module in new_net.named_modules():
    print(f"模块名称: {name}, 类型: {type(module).__name__}")

torch.Size([400, 1])
模块名称: , 类型: GCNGRU
模块名称: gcn1, 类型: GCNCov
模块名称: gcn1.linear, 类型: Linear
模块名称: gcn1.dropout, 类型: Dropout
模块名称: gcn2, 类型: GCNCov
模块名称: gcn2.linear, 类型: Linear
模块名称: gcn2.dropout, 类型: Dropout
模块名称: gru, 类型: GRU
模块名称: fc, 类型: Linear


In [7]:
x = torch.arange(0, 8, dtype=torch.float32).reshape(4, 2)
y = torch.arange(2, 10, dtype=torch.float32).reshape(4, 2)
z = torch.rand((4, 2))
def get_distance(data, curret_sample):
    """计算窗口样本的距离之和"""
    close_fn = np.abs(curret_sample - data)
    close_fn = np.exp(-1* close_fn)
    close_fn = close_fn.sum() / data.shape
    distance = 1 / (close_fn + 1e-5) -1
    return distance.item()
print(x, y)
distance, path = fastdtw(x, y, dist=get_distance)
print(distance)
print(path)
distance, path = fastdtw(x, z, dist=get_distance)
print(distance)
print(path)

tensor([[0., 1.],
        [2., 3.],
        [4., 5.],
        [6., 7.]]) tensor([[2., 3.],
        [4., 5.],
        [6., 7.],
        [8., 9.]])
12.776990315840429
[(0, 0), (1, 0), (2, 1), (3, 2), (3, 3)]
335.02376179056955
[(0, 0), (0, 1), (1, 2), (2, 3), (3, 3)]


In [3]:
def get_distance(data, curret_sample):
    """计算窗口样本的距离之和"""
    close_fn = torch.abs(curret_sample - data)
    close_fn = torch.exp(-1* close_fn)
    close_fn = close_fn.sum(dim=1) / data.shape[1]
    distance = torch.sum(1 / (close_fn + 1e-5) -1, dim=0, keepdim=True)
    return distance
distance = get_distance(x, y)
print(distance)
distance = get_distance(x, z)
print(distance)

tensor([25.5540])
tensor([453.7448])


In [4]:
a = np.zeros((2, 3))
b = np.ones((2, 3))
c = np.array([a, b])
c.reshape(c.shape[0], -1)
d = torch.tensor(0.1)
e = torch.tensor(0.2)
torch.stack((d, e))

tensor([0.1000, 0.2000])